# Kaggle Housing dataset prediction

The following notebook is a ensembling/stacking model for predicting housing sale price for kaggle competition.
On the kaggle public scoreboard, the submission scored 0.12207 being around top 20% on the leader board.

In [0]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Updating and installing latest xgboost and lightgbm packages

In [0]:
!pip install xgboost
!pip install lightgbm

In [0]:
#read data
train_set = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/house/data/train-1.csv')
test_set = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/house/data/test-1.csv')

## Data processing and feature engineering

Year data were categorized with different time buckets based on the mean and standard deviation. Some numeric data such as YrSold and MoSold were converted to categorical features since their values were limited to 10~20 discrete values. Furthermore, features that indicate degree of quality were all converted to numeric data. Credits to Kaggle Kernel "Comprehensive data exploration with Python" for data visualization/feature engineering methodology

In [0]:
from math import isnan
from sklearn.preprocessing import RobustScaler

def preprocess_data(data):
    description = data.describe(include='all')
    for column in data.columns:
        #if categorical data
        if isnan(description.loc["mean"][column]):
            #fill missing values with top categorical value
            data[column] = data[column].fillna(description.loc["top"][column])
        #bucketize timezones
        elif column=="GarageYrBlt":
            data[column] = pd.cut(data[column],[1894.885,1918,1941,1964,1987,2010])
        elif column=="YearBuilt":
            data[column] = pd.cut(data[column],[1871.862,1899.6,1927.2,1954.8,1982.4,2010])
        elif column=="YearRemodAdd":
            data[column] = pd.cut(data[column],[1949.94,1962.0,1974,1986,1998,2010])
        else:
        #otherwise fill missing data with mean for numeric data
            data[column] = data[column].fillna(description.loc["mean"][column])
    #convert numeric data to categorical data
    data['MSSubClass'] = data['MSSubClass'].apply(lambda x: str(x))
    data['YrSold'] = data['YrSold'].apply(lambda x: str(x))
    data['MoSold'] = data['MoSold'].apply(lambda x: str(x))
    
    #drop irrelevant data.
    #Id: index
    #PoolQC, Street, Alley, Utilities, MiscFeature: too sparse for relevant analysis
    data = data.drop(['PoolQC','Street','Alley','Utilities','MiscFeature','Id'],axis=1)
    
    #convert categorical data to numberic data
    quality_mapping = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NA':0}
    exposure_mapping = {'Gd':4,'Av':3,'Mn':2,'No':1,'NA':0}
    finType_mapping = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NA':0}
    data = data.replace({'HouseStyle':
                            {'1Story':1,'1.5Fin':1.5,'1.5Unf':1.25,'2Story':2,'2.5Fin':2.5,'2.5Unf':2.25,'SFoyer':3,'SLvl':3},
                         'ExterCond':quality_mapping,
                         'ExterQual':quality_mapping,
                         'BsmtQual':quality_mapping,
                         'BsmtCond':quality_mapping,
                         'BsmtExposure':exposure_mapping,
                         'BsmtFinType1':finType_mapping,
                         'BsmtFinType2':finType_mapping,
                         'HeatingQC':quality_mapping,
                         'KitchenQual':quality_mapping,
                         'FireplaceQu':quality_mapping,
                         'GarageFinish':{'Fin':3,'RFn':2,'Unf':1,'NA':0},
                         'GarageQual':quality_mapping,
                         'GarageCond':quality_mapping,
                        })
    #one-hot encode data
    data = pd.get_dummies(data)        
    return data

In [0]:
#process train and test set together
predictions_set = test_set.loc[:,['Id']]
dataset = pd.concat([train_set,test_set])
dataset = preprocess_data(dataset)

#split into train and test set again
train_set = dataset.iloc[0:1460]
test_features = dataset.iloc[1460:]
test_features.pop('SalePrice')

0       180921.19589
1       180921.19589
2       180921.19589
3       180921.19589
4       180921.19589
5       180921.19589
6       180921.19589
7       180921.19589
8       180921.19589
9       180921.19589
10      180921.19589
11      180921.19589
12      180921.19589
13      180921.19589
14      180921.19589
15      180921.19589
16      180921.19589
17      180921.19589
18      180921.19589
19      180921.19589
20      180921.19589
21      180921.19589
22      180921.19589
23      180921.19589
24      180921.19589
25      180921.19589
26      180921.19589
27      180921.19589
28      180921.19589
29      180921.19589
            ...     
1429    180921.19589
1430    180921.19589
1431    180921.19589
1432    180921.19589
1433    180921.19589
1434    180921.19589
1435    180921.19589
1436    180921.19589
1437    180921.19589
1438    180921.19589
1439    180921.19589
1440    180921.19589
1441    180921.19589
1442    180921.19589
1443    180921.19589
1444    180921.19589
1445    18092

In [0]:
#code for visualization. Creates giant heatmap and pairplot analysis for the train set. Commented to save computation. 
#View corr.png and pairplot.png files in the directory for the result

#fig, ax = plt.subplots(figsize=(100,100)) 
#sns.heatmap(data.corr(),ax=ax,annot_kws={'size': 100}).get_figure().savefig("./corr.png")
#g = sns.PairGrid(data)
#g = g.map(plt.scatter)
#g.savefig("./pairplot.png")

## Removing Outliers

Now that we have imputed all missing data and done some feature engineering, let's make sure that we get rid of any outliers which might undermine our model

In [0]:
#dictionary for outlier ranges. Outlier values determined from pairplot graph above
outliers = {
    'LotFrontage': 250,
    'LotArea': 15000,
    'BsmtFinSF1':3000,
    'TotalBsmtSF':4000,
    '1stFlrSF':4000,
    'LowQualFinSF':800,
    'WoodDeckSF':1250,
    'EnclosedPorch':800,
}
#remove data that are out of range
def remove_outlier(data):
    for column, drop_cutoff in outliers.items():
        data = data.drop(data[data[column]>drop_cutoff].index)
    return data

In [0]:
#Optionally use EllipticEnvelope for outlier removal. EllipticEnvelope identifies outliers based on data's distance from mean scaled by std.

#from sklearn.covariance import EllipticEnvelope
#def remove_outlier(data):
#    detector = EllipticEnvelope(contamination=0.05)
#    detector.fit(data.values)
#    outliers = detector.predict(data.values)
#    outliers_index = np.where(outliers==-1)[0]
#    data = data.drop(outliers_index,axis=0)
#    return data

In [0]:
#remove outliers
train_set = remove_outlier(train_set)

## Preparing Dataset for training
Since our dataset is ready to go, we can normalize our data for feature scaling to improve performance and do additional feature engineering before proceeding to model selection.

In [0]:
#use robust scaler for normalizing input and feature scaling
scaler = RobustScaler()
labels = np.log1p(train_set.pop('SalePrice').values)
train_set = pd.DataFrame(data=scaler.fit_transform(train_set),columns=train_set.columns)
test_features = scaler.fit_transform(test_features)

In [0]:
from sklearn.ensemble import RandomForestRegressor

#use random forest algorithm to determine minor features
randomForest = RandomForestRegressor(bootstrap=False,max_depth=50,max_features='sqrt',min_samples_leaf=1,min_samples_split=3,n_estimators=2000)
randomForest.fit(train_set,labels)
sorted_importances = sorted(enumerate(randomForest.feature_importances_),key=lambda x: x[1])
features_to_drop = []
features_index_to_drop = []

#drop 60 features with least feature importances
for i,importance in sorted_importances[0:60]:
    features_to_drop.append(train_set.columns[i])
    features_index_to_drop.append(i)
train_set = train_set.drop(columns=features_to_drop)
test_features = np.delete(test_features,features_index_to_drop,axis=1)
train_set.columns = list(range(0,train_set.shape[1]))
features = train_set.values

##Training the model

Since we will be stacking multiple models, we will try to choose different kinds of model raning from ensembling methods to simple linear models. We will also use XGBoost and LightGBM packages which have been known for good performance. Note that the chosen models and parameters were already tuned.

In [0]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNet, Lasso, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
import lightgbm as lgbm

#uncomment if you wish to add different models, but ElasticNet, KernelRidge, and LightGBM models have shown better performance overall.
classifiers = {
    'ElasticNet':ElasticNet(alpha=0.0005,l1_ratio=0.9),
    #'adaBoost':AdaBoostRegressor(n_estimators=3200,learning_rate=0.01),
    'gradientBoosting':GradientBoostingRegressor(learning_rate=0.02,loss='huber',max_depth=3,min_samples_leaf=4,min_samples_split=9,n_estimators=3000,subsample=0.9),
    #'ex_tree':ExtraTreesRegressor(bootstrap=False,max_depth=50,max_features='auto',min_samples_leaf=2,min_samples_split=3,n_estimators=1800,n_jobs=4),
    'ridge':KernelRidge(alpha=0.1,coef0=6,degree=1,kernel='polynomial'),
    #'xgboost':XGBRegressor(colsample_bytree=0.3,gamma=0.1,learning_rate=0.01,max_depth=60,min_child_weight=3,n_estimators=3000,subsample=0.4),
    'lgbm': lgbm.sklearn.LGBMRegressor(boosting_type='goss',colsample_bytree=0.5,learning_rate=0.005,max_depth=10,min_child_samples=10,min_child_weight=7,n_estimators=2500,n_jobs=4,num_leaves=7,reg_lambda=1,subsample=1.0),
}

In [0]:
#This section is for using RandomSearch and GridSearch for parameter tuning. Feel free to uncomment certain parameters for tuning it.

#from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_score, GridSearchCV
#params = {
        #'alpha':[1.0,0.5,0.1,0.05,0.001,0.005,0.0001,0.0005,0.00001,0.00005],
        #'l1_ratio':[0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0],
        #'normalize':[True,False]
        #'n_estimators': [400,500,600,700,900,1200,1500,1800,2000,2300,2500,3000,3200,3500,4000],
        #'learning_rate': [0.1,0.01,0.02,0.001,0.005,0.0001,0.0005,0.00001],
        #'min_child_weight': [0.00001,0.0001,0.001,0.01,0.1,1,3,5,7],
        #'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
        #'max_depth': [8,9,10,15,20,30,40,50,60,70],
        #'depth':[3,1,2,6,4,5,7,8,9,10],
        #'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
        #'num_leaves':[7,14,21,31,50,100,150,200,300,400,500,1000,1500,2000,2500,3000,3500,4000,4500],
        #'boosting_type':['gbdt','goss','dart'],
        #'min_child_samples':[5,10,15,20,25,30,40,50,70,90],
        #'max_features': ['auto','sqrt'],
        #'min_samples_split':[2,3,5,7,9,11,13,15],
        #'min_samples_leaf':[1,2,4,6,9,11,13,15],
        #'bootstrap':[True,False],
        #'subsample':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
        #'reg_lambda':[1,0,0.1,0.001,0.0001,0.00001,0.000001],
        #'loss':['ls', 'lad', 'huber', 'quantile'],
        #'kernel':['rbf','linear','polynomial','sigmoid'],
        #'degree':[1,2,3,4,5,6,7,8,9,10],
        #'coef0':[1,2,3,4,5,6,7,8]
        #'C':[0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
        #'l2_leaf_reg':[3,1,5,10,20,30,50,70,90,100,120,150],
        #'iterations':[250,100,500,700,1000,1500],
        #'bagging_temperature':[7,14,21,31,50,100,150,200,300,400,500,1000]
#}
#model = XGBRegressor(n_jobs=4)
#kf = KFold(n_splits=4,shuffle=True)
"""
random_search = RandomizedSearchCV(model,
                                   param_distributions=params,
                                   n_iter=1500,
                                   scoring='neg_mean_squared_error',
                                   cv=skf.split(features,labels),
                                   verbose=3, 
                                   n_jobs=4)
"""
#random_search = GridSearchCV(model,param_grid=params,cv=kf.split(features,labels),verbose=3,n_jobs=4)
#random_search.fit(features,labels)

"\nrandom_search = RandomizedSearchCV(model,\n                                   param_distributions=params,\n                                   n_iter=1500,\n                                   scoring='neg_mean_squared_error',\n                                   cv=skf.split(features,labels),\n                                   verbose=3, \n                                   n_jobs=4)\n"

In [0]:
#print parameter search results

"""
print(random_search.cv_results_)
print(random_search.best_estimator_.score(features,labels))
print(random_search.best_score_)
print(random_search.best_estimator_)
"""

'\nprint(random_search.cv_results_)\nprint(random_search.best_estimator_.score(features,labels))\nprint(random_search.best_score_)\nprint(random_search.best_estimator_)\n'

Before getting our first round of base model predictions, let's make sure that these are good models to try out using KFold validation. Note that Kaggle has a seperate dataset for just scoring the submissions so using heavy validation methods such as KFold validation is highly recommended.

In [0]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
kf = KFold(n_splits=4,shuffle=True)

#get KFold validation results
def get_score(model):
    score = np.sqrt(-cross_val_score(model,features,labels,scoring="neg_mean_squared_error",cv=kf))
    return score

In [0]:
for key, classifier in classifiers.items():
    print(key,get_score(classifier).mean())

ElasticNet 0.11395888261874398
gradientBoosting 0.11284385430876491
ridge 0.11305002544611711
lgbm 0.11930848511827917


The results seem really promissing with all of the validation loss lower than 0.12 benchmark.

##Base model predictions

Here, we will use each models to print 1/n of the predictions of the whole dataset using KFold. The base model predictions on train set will be concatenated for our second round of predictions with stacking. The test set predictions will be averaged for prediciton on the second round as well. View **xgboost-house** notebook to see how stacking is done with XGBoost model.

In [0]:
for i,(train_index,test_index) in enumerate(kf.split(features)):
    keys = list(classifiers.keys())
    classifier = classifiers[keys[i]]
    X_train, y_train = features[train_index], labels[train_index]
    X_test, y_test = features[test_index], labels[test_index]
    classifier.fit(X_train,y_train)
    
    #get model score and loss for the fold
    print(keys[i],np.sqrt(mean_squared_error(classifier.predict(X_test),y_test)))
    print(keys[i],classifier.score(X_test,y_test))
    
    #get features for stacking
    stacking_features = classifier.predict(X_test)
    stacking_set = pd.DataFrame({'Id':test_index})
    stacking_set = stacking_set.assign(SalePrice=pd.Series(stacking_features))
    stacking_set.to_csv('/content/gdrive/My Drive/Colab Notebooks/house/stacking_model/'+keys[i]+'.csv',index=False)
    
    #get predictions from test_features
    predictions = classifier.predict(test_features)
    predictions_set = predictions_set.assign(SalePrice=pd.Series(predictions))
    predictions_set.to_csv('/content/gdrive/My Drive/Colab Notebooks/house/saved_model/'+keys[i]+'.csv',index=False)
#save labels for later reference
pd.Series(labels).to_csv('/content/gdrive/My Drive/Colab Notebooks/house/saved_model/labels.csv',index=False,header=True)

ElasticNet 0.1024708319306177
ElasticNet 0.9344335732239912
gradientBoosting 0.12812819743845516
gradientBoosting 0.8806605759924498
ridge 0.10527682928125777
ridge 0.919638729597537
lgbm 0.1201408254126671
lgbm 0.9117547586448226
